In [1]:
from pyspark.sql import functions as F
data= [[1,'1970-01-01 07:00:00','1970-01-01 07:03:00'],
       [2,'1970-01-01 07:49:00','1970-01-01 07:54:00']]
columns= ['id','starttime','endtime']
df= spark.createDataFrame(data,columns)
df.show()
df.printSchema()

+---+-------------------+-------------------+
 id| starttime| endtime|
+---+-------------------+-------------------+
 1|1970-01-01 07:00:00|1970-01-01 07:03:00|
 2|1970-01-01 07:49:00|1970-01-01 07:54:00|
+---+-------------------+-------------------+

root
-- id: long (nullable = true)
-- starttime: string (nullable = true)
-- endtime: string (nullable = true)

In [3]:
df1=df.withColumn("starttime", F.col("starttime").cast("timestamp")).withColumn("endtime", F.col("endtime").cast('timestamp'))
df1.show()

+---+-------------------+-------------------+
 id| starttime| endtime|
+---+-------------------+-------------------+
 1|1970-01-01 07:00:00|1970-01-01 07:03:00|
 2|1970-01-01 07:49:00|1970-01-01 07:54:00|
+---+-------------------+-------------------+

In [4]:
df1.select('id', F.explode(F.expr('sequence(starttime, endtime, interval 1 minute)')).alias('starttime'), 'endtime' ).show(truncate=False)

+---+-------------------+-------------------+
id |starttime |endtime |
+---+-------------------+-------------------+
1 |1970-01-01 07:00:00|1970-01-01 07:03:00|
1 |1970-01-01 07:01:00|1970-01-01 07:03:00|
1 |1970-01-01 07:02:00|1970-01-01 07:03:00|
1 |1970-01-01 07:03:00|1970-01-01 07:03:00|
2 |1970-01-01 07:49:00|1970-01-01 07:54:00|
2 |1970-01-01 07:50:00|1970-01-01 07:54:00|
2 |1970-01-01 07:51:00|1970-01-01 07:54:00|
2 |1970-01-01 07:52:00|1970-01-01 07:54:00|
2 |1970-01-01 07:53:00|1970-01-01 07:54:00|
2 |1970-01-01 07:54:00|1970-01-01 07:54:00|
+---+-------------------+-------------------+

In [5]:
df2=df1.withColumn("diff", F.minute("endtime")-F.minute("starttime")).withColumn("end", F.expr("array_repeat(endtime, diff+1)"))
df2.show()

+---+-------------------+-------------------+----+--------------------+
 id| starttime| endtime|diff| end|
+---+-------------------+-------------------+----+--------------------+
 1|1970-01-01 07:00:00|1970-01-01 07:03:00| 3|[1970-01-01 07:03...|
 2|1970-01-01 07:49:00|1970-01-01 07:54:00| 5|[1970-01-01 07:54...|
+---+-------------------+-------------------+----+--------------------+

In [6]:
w=Window().partitionBy("id").orderBy("starttime")
w2=Window().partitionBy("id2").orderBy("starttime")
df2.withColumn("end",F.explode("end")).withColumn("num", F.row_number().over(w)).withColumn("id2", F.when(F.col("num")==1, F.lit(-1)).otherwise(F.col("id"))).withColumn("num2",F.when(F.col("id2")==-1, F.lit(0)).otherwise(F.row_number().over(w2))).withColumn("start",(F.unix_timestamp("starttime")+F.col("num2")*60).cast('timestamp'))\
.select("id",F.col("start").alias("starttime"),F.col("end").alias("endtime"))\
.orderBy(F.col("id"),F.col("starttime").asc()).show()

+---+-------------------+-------------------+
 id| starttime| endtime|
+---+-------------------+-------------------+
 1|1970-01-01 07:00:00|1970-01-01 07:03:00|
 1|1970-01-01 07:01:00|1970-01-01 07:03:00|
 1|1970-01-01 07:02:00|1970-01-01 07:03:00|
 1|1970-01-01 07:03:00|1970-01-01 07:03:00|
 2|1970-01-01 07:49:00|1970-01-01 07:54:00|
 2|1970-01-01 07:50:00|1970-01-01 07:54:00|
 2|1970-01-01 07:51:00|1970-01-01 07:54:00|
 2|1970-01-01 07:52:00|1970-01-01 07:54:00|
 2|1970-01-01 07:53:00|1970-01-01 07:54:00|
 2|1970-01-01 07:54:00|1970-01-01 07:54:00|
+---+-------------------+-------------------+

In [7]:
+---+-------------------+-------------------+
| id|          starttime|            endtime|
+---+-------------------+-------------------+
|  1|1970-01-01 07:00:00|1970-01-01 07:03:00|
+---+-------------------+-------------------+
|  1|1970-01-01 07:01:00|1970-01-01 07:03:00|
+---+-------------------+-------------------+
|  1|1970-01-01 07:02:00|1970-01-01 07:03:00|
+---+-------------------+-------------------+
|  1|1970-01-01 07:03:00|1970-01-01 07:03:00|
+---+-------------------+-------------------+

In [8]:
data= [[ 'abc.com','_spf.a22.biz'],
       [ 'abc.com','_spf.google.com'],
       ['_spf.google.com','_spf.mail.ru']]
columns= ['src','dst']
df= spark.createDataFrame(data,columns)
df.show()

+---------------+---------------+
 src| dst|
+---------------+---------------+
 abc.com| _spf.a22.biz|
 abc.com|_spf.google.com|
_spf.google.com| _spf.mail.ru|
+---------------+---------------+

In [9]:
expression="""filter(both, x -> x != '_spf' )"""
df.withColumn("src1",F.split("src","\\.")).withColumn("dst1", F.split("dst","\\.")).withColumn("both",F.array_intersect("src1","dst1")).withColumn("both1",F.expr(expression)).withColumn("linkage_count",F.size("both1")).drop("src1","dst1","both","both1").show()

+---------------+---------------+-------------+
 src| dst|linkage_count|
+---------------+---------------+-------------+
 abc.com| _spf.a22.biz| 0|
 abc.com|_spf.google.com| 1|
_spf.google.com| _spf.mail.ru| 0|
+---------------+---------------+-------------+

In [10]:
df.withColumn("src1",F.split("src","\\.")).withColumn("dst1", F.split("dst","\\.")).withColumn("both",F.array_intersect("src1","dst1")).withColumn("linkage_count",F.size("both")).drop("src1","dst1","both").show()

+---------------+---------------+-------------+
 src| dst|linkage_count|
+---------------+---------------+-------------+
 abc.com| _spf.a22.biz| 0|
 abc.com|_spf.google.com| 1|
_spf.google.com| _spf.mail.ru| 1|
+---------------+---------------+-------------+